In [1]:
# Import packages
# dxpy allows python to interact with the platform storage
# Note: This notebook is using spark since the size of the dataset we're extracting
# (i.e. the number of fields) is too large for a single node instance.
import dxdata
import dxpy
import pandas as pd
import subprocess
import glob
import os
import pyspark
from pyspark import SparkConf, SparkContext
from pyspark.sql import SQLContext

In [2]:
import numpy as np

In [7]:
olink1 = pd.read_csv('olink_data_ready1_for_pravesh.csv',sep='\t')
olink1.head()

,FID,IID,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,...,ZFYVE19,ZHX2,ZNF174,ZNF75D,ZNF830,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1
0,1000028,1000028,0.3140,0.06345,-0.40685,0.5843,-0.10025,0.16690,1.4302,-0.4886,...,-0.2795,0.3685,-0.2194,-0.1784,-0.14260,0.70005,0.2462,2.0002,-0.4964,0.7369
1,1000045,1000045,-0.0512,-0.18295,0.17320,-0.1432,-0.19310,-0.45110,1.2723,-0.0085,...,-0.3825,0.1917,0.0727,-0.0292,0.29005,-0.00320,-0.2965,-0.3734,0.0405,-0.1825
2,1000246,1000246,NaN,NaN,-0.90395,NaN,1.10775,-0.26705,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1000624,1000624,0.2874,-0.17145,0.42140,-0.0853,-0.25490,-0.74020,-2.1225,0.0602,...,-0.4211,0.3927,0.3814,-0.6578,0.04785,-0.04580,-0.0013,-4.5999,0.2328,0.1268
4,1000950,1000950,NaN,NaN,1.25435,NaN,-0.08715,0.38445,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
!dx download olink_data_ready1_for_pravesh.csv

[===========================================================>] Completed 1,112,172,458 of 1,112,172,458 bytes (100%) /opt/notebooks/olink_data_ready1_for_pravesh.csvv


In [8]:
olink1.shape

(53013, 2925)

In [16]:
!dx download /zillur/bip_bmi_smd/pheno/caucasian*

[===========================================================>] Completed 42,189,248 of 42,189,248 bytes (100%) /opt/notebooks/caucasian_age_sex_10pc_lifestyle.txtt
[===========================================================>] Completed 8,332,418 of 8,332,418 bytes (100%) /opt/notebooks/caucasian_bmi_norelative.txtt


In [14]:
prot1 = pd.read_csv('proteome_md_bmi_physact.csv',sep='\t')
prot1.head()

,FID,IID,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,...,ZNF830,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1,BIP,MD,Obesity,PhysAct
0,1000045,1000045,-0.051200,-0.182950,0.17320,-0.14320,-0.19310,-0.45110,1.272300,-0.008500,...,0.290050,-0.003200,-0.296500,-0.373400,0.04050,-0.182500,0,0,0,1
1,1000246,1000246,-0.004295,0.021876,-0.90395,0.01250,1.10775,-0.26705,-0.168134,0.041187,...,0.256078,0.086721,0.029102,-1.090898,0.03333,0.218336,0,0,0,0
2,1000624,1000624,0.287400,-0.171450,0.42140,-0.08530,-0.25490,-0.74020,-2.122500,0.060200,...,0.047850,-0.045800,-0.001300,-4.599900,0.23280,0.126800,0,0,0,1
3,1000950,1000950,-0.004295,0.021876,1.25435,0.01250,-0.08715,0.38445,-0.168134,0.041187,...,0.256078,0.086721,0.029102,-1.090898,0.03333,0.218336,0,0,0,0
4,1002400,1002400,0.048400,0.109050,0.71500,0.21765,-0.09530,0.45640,0.524700,0.229300,...,-0.437700,0.335650,0.069700,-0.511600,-0.19650,-0.438900,0,0,1,0


In [17]:
cauc1 = pd.read_csv('caucasian_bmi_norelative.txt',sep='\t')
cauc1.head()

,FID,IID,BMI
0,1000010,1000010,19.8964
1,1000028,1000028,28.8015
2,1000045,1000045,21.7982
3,1000052,1000052,45.2910
4,1000076,1000076,30.0143


In [18]:
olink2 = olink1[olink1.FID.isin(cauc1.FID)]
olink2.shape

(37356, 2925)

In [20]:
olink3 = olink2.merge(cauc1,on=['FID','IID'])
olink3.head()

,FID,IID,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,...,ZHX2,ZNF174,ZNF75D,ZNF830,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1,BMI
0,1000028,1000028,0.3140,0.06345,-0.40685,0.5843,-0.10025,0.16690,1.4302,-0.4886,...,0.3685,-0.2194,-0.1784,-0.14260,0.70005,0.2462,2.0002,-0.4964,0.7369,28.8015
1,1000045,1000045,-0.0512,-0.18295,0.17320,-0.1432,-0.19310,-0.45110,1.2723,-0.0085,...,0.1917,0.0727,-0.0292,0.29005,-0.00320,-0.2965,-0.3734,0.0405,-0.1825,21.7982
2,1000246,1000246,NaN,NaN,-0.90395,NaN,1.10775,-0.26705,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.7118
3,1000624,1000624,0.2874,-0.17145,0.42140,-0.0853,-0.25490,-0.74020,-2.1225,0.0602,...,0.3927,0.3814,-0.6578,0.04785,-0.04580,-0.0013,-4.5999,0.2328,0.1268,21.9928
4,1000950,1000950,NaN,NaN,1.25435,NaN,-0.08715,0.38445,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.7685


In [21]:
olink3['Obesity'] = np.where(olink3.BMI >= 30, 1, 0)
olink3.head()

,FID,IID,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,...,ZNF174,ZNF75D,ZNF830,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1,BMI,Obesity
0,1000028,1000028,0.3140,0.06345,-0.40685,0.5843,-0.10025,0.16690,1.4302,-0.4886,...,-0.2194,-0.1784,-0.14260,0.70005,0.2462,2.0002,-0.4964,0.7369,28.8015,0
1,1000045,1000045,-0.0512,-0.18295,0.17320,-0.1432,-0.19310,-0.45110,1.2723,-0.0085,...,0.0727,-0.0292,0.29005,-0.00320,-0.2965,-0.3734,0.0405,-0.1825,21.7982,0
2,1000246,1000246,NaN,NaN,-0.90395,NaN,1.10775,-0.26705,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.7118,0
3,1000624,1000624,0.2874,-0.17145,0.42140,-0.0853,-0.25490,-0.74020,-2.1225,0.0602,...,0.3814,-0.6578,0.04785,-0.04580,-0.0013,-4.5999,0.2328,0.1268,21.9928,0
4,1000950,1000950,NaN,NaN,1.25435,NaN,-0.08715,0.38445,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,28.7685,0


In [22]:
olink3.Obesity.value_counts()

Obesity
0    28303
1     9053
Name: count, dtype: int64

In [24]:
olink4 = olink3.drop('BMI',axis=1)
olink4.head()

,FID,IID,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,...,ZHX2,ZNF174,ZNF75D,ZNF830,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1,Obesity
0,1000028,1000028,0.3140,0.06345,-0.40685,0.5843,-0.10025,0.16690,1.4302,-0.4886,...,0.3685,-0.2194,-0.1784,-0.14260,0.70005,0.2462,2.0002,-0.4964,0.7369,0
1,1000045,1000045,-0.0512,-0.18295,0.17320,-0.1432,-0.19310,-0.45110,1.2723,-0.0085,...,0.1917,0.0727,-0.0292,0.29005,-0.00320,-0.2965,-0.3734,0.0405,-0.1825,0
2,1000246,1000246,NaN,NaN,-0.90395,NaN,1.10775,-0.26705,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
3,1000624,1000624,0.2874,-0.17145,0.42140,-0.0853,-0.25490,-0.74020,-2.1225,0.0602,...,0.3927,0.3814,-0.6578,0.04785,-0.04580,-0.0013,-4.5999,0.2328,0.1268,0
4,1000950,1000950,NaN,NaN,1.25435,NaN,-0.08715,0.38445,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0


In [25]:
lf1 = pd.read_csv('caucasian_age_sex_10pc_lifestyle.txt',sep='\t')
lf1.head()

,FID,IID,Sex,Age,PC1,PC2,PC3,PC4,PC5,PC6,...,Beef_intake,Oily_fish_intake,Non_oily_fish_intake,Processed_meat_intake,Poultry_intake,Lam_or_mutton_intake,Cooked_vegetable_intake,Salad_or_raw_vegetable_intake,Fresh_fruit,Dried_fruit
0,1000010,1000010,0,50,-15.4208,4.06047,-1.314790,4.13000,-8.97314,0.532284,...,0.0,1.0,1.0,0.0,3.0,0.0,3,2,1,0
1,1000045,1000045,1,62,-11.5385,3.69257,-3.597900,3.87470,-9.12039,-2.828620,...,1.0,3.0,1.0,3.0,0.0,0.0,1,0,2,0
2,1000052,1000052,1,56,-11.1988,7.25941,-1.117110,-1.93459,-3.73551,-2.270200,...,1.0,3.0,3.0,1.0,3.0,1.0,6,6,3,2
3,1000076,1000076,0,55,-15.2215,2.99643,-0.875316,4.54313,9.41637,1.974870,...,1.0,1.0,1.0,0.0,1.0,0.0,5,2,2,0
4,1000091,1000091,0,55,-11.3446,3.78242,-1.846780,2.04100,8.58898,0.895529,...,1.0,3.0,1.0,0.0,3.0,1.0,3,2,3,0


In [26]:
lf1.columns

Index(['FID', 'IID', 'Sex', 'Age', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'PC6',
       'PC7', 'PC8', 'PC9', 'PC10', 'MVPA', 'LST', 'Beef_intake',
       'Oily_fish_intake', 'Non_oily_fish_intake', 'Processed_meat_intake',
       'Poultry_intake', 'Lam_or_mutton_intake', 'Cooked_vegetable_intake',
       'Salad_or_raw_vegetable_intake', 'Fresh_fruit', 'Dried_fruit'],
      dtype='object')

In [27]:
lf1.shape

(318113, 26)

In [28]:
lf1.MVPA.value_counts()

MVPA
0    165997
1    152116
Name: count, dtype: int64

In [30]:
olink5 = olink4.merge(lf1[['FID','IID','MVPA']], on = ['FID','IID'])
olink5.head()

,FID,IID,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,...,ZNF174,ZNF75D,ZNF830,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1,Obesity,MVPA
0,1000045,1000045,-0.0512,-0.18295,0.17320,-0.14320,-0.19310,-0.45110,1.2723,-0.0085,...,0.0727,-0.0292,0.29005,-0.00320,-0.2965,-0.3734,0.0405,-0.1825,0,1
1,1000246,1000246,NaN,NaN,-0.90395,NaN,1.10775,-0.26705,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,1000624,1000624,0.2874,-0.17145,0.42140,-0.08530,-0.25490,-0.74020,-2.1225,0.0602,...,0.3814,-0.6578,0.04785,-0.04580,-0.0013,-4.5999,0.2328,0.1268,0,1
3,1000950,1000950,NaN,NaN,1.25435,NaN,-0.08715,0.38445,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,1002400,1002400,0.0484,0.10905,0.71500,0.21765,-0.09530,0.45640,0.5247,0.2293,...,0.0831,0.0466,-0.43770,0.33565,0.0697,-0.5116,-0.1965,-0.4389,1,0


In [31]:
olink5.MVPA.value_counts()

MVPA
0    15286
1    13476
Name: count, dtype: int64

In [32]:
olink6 = olink5.rename(columns={'MVPA':'PhysAct'})
olink6.head()

,FID,IID,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,...,ZNF174,ZNF75D,ZNF830,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1,Obesity,PhysAct
0,1000045,1000045,-0.0512,-0.18295,0.17320,-0.14320,-0.19310,-0.45110,1.2723,-0.0085,...,0.0727,-0.0292,0.29005,-0.00320,-0.2965,-0.3734,0.0405,-0.1825,0,1
1,1000246,1000246,NaN,NaN,-0.90395,NaN,1.10775,-0.26705,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
2,1000624,1000624,0.2874,-0.17145,0.42140,-0.08530,-0.25490,-0.74020,-2.1225,0.0602,...,0.3814,-0.6578,0.04785,-0.04580,-0.0013,-4.5999,0.2328,0.1268,0,1
3,1000950,1000950,NaN,NaN,1.25435,NaN,-0.08715,0.38445,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0
4,1002400,1002400,0.0484,0.10905,0.71500,0.21765,-0.09530,0.45640,0.5247,0.2293,...,0.0831,0.0466,-0.43770,0.33565,0.0697,-0.5116,-0.1965,-0.4389,1,0


In [33]:
# Automatically discover dispensed dataset ID
dispensed_dataset = dxpy.find_one_data_object(
    typename="Dataset", name="app*.dataset", folder="/", name_mode="glob"
)
dispensed_dataset_id = dispensed_dataset["id"]

In [34]:
# Get project ID
project_id = dxpy.find_one_project()["id"]

In [35]:
dataset = (":").join([project_id, dispensed_dataset_id])

In [36]:
# Note: This cell can only be run once. Otherwise, you'll need to delete the existing data tables in order to re-run
cmd = ["dx", "extract_dataset", dataset, "-ddd", "--delimiter", ","]
subprocess.check_call(cmd)

0

In [37]:
path = os.getcwd()

In [38]:
# Need to adjust this buffer otherwise will get an error in toPandas() call
conf = pyspark.SparkConf().set("spark.kryoserializer.buffer.max", "256m")

In [39]:
sc = pyspark.SparkContext(conf=conf)
spark = pyspark.sql.SparkSession(sc)
sqlContext = SQLContext(sc)

/cluster/spark/python/pyspark/sql/context.py:113: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [40]:
# Load the original dataset - record-id can be obtained from inspecting details of the database in the UI
dataset = dxdata.load_dataset(id='record-Gq77990JqZXKQB0PzqKXfXxy')

In [42]:
# Load your pre-filtered cohort
cont = dxdata.load_cohort(folder="/", name="bmi_all_sample_pheno")

In [43]:
participant = dataset["participant"]

In [44]:
fields = ['eid','p20126_i0']

In [45]:
cont_df = participant.retrieve_fields(names=fields, filter_sql=cont.sql, engine=dxdata.connect()).toPandas()
cont_df.head()

/opt/conda/lib/python3.11/site-packages/thrift/transport/TSSLSocket.py:53: DeprecationWarning: ssl.PROTOCOL_TLS is deprecated
  self._context = ssl.SSLContext(ssl_version)


,eid,p20126_i0
0,1000028,NaN
1,1000045,NaN
2,1000104,NaN
3,1000171,NaN
4,1000196,0.0


In [49]:
cont_df['MD'] = np.where(cont_df.p20126_i0.isin([3,4,5]), 1, 0)
cont_df.head()

,eid,p20126_i0,MD
0,1000028,NaN,0
1,1000045,NaN,0
2,1000104,NaN,0
3,1000171,NaN,0
4,1000196,0.0,0


In [50]:
cont_df.MD.value_counts()

MD
0    467455
1     31581
Name: count, dtype: int64

In [53]:
olink6['FID'] = olink6['FID'].astype(str)

# If eid should be a number:
cont_df['eid'] = cont_df['eid'].astype(str)

In [54]:
olink7 = pd.merge(olink6,cont_df[['eid','MD']],left_on = 'FID',right_on = 'eid')
olink7.head()

,FID,IID,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,...,ZNF830,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1,Obesity,PhysAct,eid,MD
0,1000045,1000045,-0.0512,-0.18295,0.17320,-0.14320,-0.19310,-0.45110,1.2723,-0.0085,...,0.29005,-0.00320,-0.2965,-0.3734,0.0405,-0.1825,0,1,1000045,0
1,1000246,1000246,NaN,NaN,-0.90395,NaN,1.10775,-0.26705,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1000246,0
2,1000624,1000624,0.2874,-0.17145,0.42140,-0.08530,-0.25490,-0.74020,-2.1225,0.0602,...,0.04785,-0.04580,-0.0013,-4.5999,0.2328,0.1268,0,1,1000624,0
3,1000950,1000950,NaN,NaN,1.25435,NaN,-0.08715,0.38445,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,1000950,0
4,1002400,1002400,0.0484,0.10905,0.71500,0.21765,-0.09530,0.45640,0.5247,0.2293,...,-0.43770,0.33565,0.0697,-0.5116,-0.1965,-0.4389,1,0,1002400,0


In [55]:
olink8 = olink7.drop('eid',axis=1)
olink8.head()

,FID,IID,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,...,ZNF75D,ZNF830,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1,Obesity,PhysAct,MD
0,1000045,1000045,-0.0512,-0.18295,0.17320,-0.14320,-0.19310,-0.45110,1.2723,-0.0085,...,-0.0292,0.29005,-0.00320,-0.2965,-0.3734,0.0405,-0.1825,0,1,0
1,1000246,1000246,NaN,NaN,-0.90395,NaN,1.10775,-0.26705,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
2,1000624,1000624,0.2874,-0.17145,0.42140,-0.08530,-0.25490,-0.74020,-2.1225,0.0602,...,-0.6578,0.04785,-0.04580,-0.0013,-4.5999,0.2328,0.1268,0,1,0
3,1000950,1000950,NaN,NaN,1.25435,NaN,-0.08715,0.38445,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0
4,1002400,1002400,0.0484,0.10905,0.71500,0.21765,-0.09530,0.45640,0.5247,0.2293,...,0.0466,-0.43770,0.33565,0.0697,-0.5116,-0.1965,-0.4389,1,0,0


In [56]:
olink8.Obesity.value_counts()

Obesity
0    22098
1     6582
Name: count, dtype: int64

In [57]:
olink8.PhysAct.value_counts()

PhysAct
0    15235
1    13445
Name: count, dtype: int64

In [58]:
olink8.MD.value_counts()

MD
0    26825
1     1855
Name: count, dtype: int64

In [59]:
olink8.shape

(28680, 2928)

In [60]:
olink8.PhysAct.isna().sum()

0

In [62]:
lf1['FID'] = lf1['FID'].astype(str)

In [63]:
olink9 = olink8.merge(lf1[['FID','Sex','Age']], on = 'FID')
olink9.head()

,FID,IID,A1BG,AAMDC,AARSD1,ABCA2,ABHD14B,ABL1,ABO,ABRAXAS2,...,ZNRD2,ZNRF4,ZP3,ZP4,ZPR1,Obesity,PhysAct,MD,Sex,Age
0,1000045,1000045,-0.0512,-0.18295,0.17320,-0.14320,-0.19310,-0.45110,1.2723,-0.0085,...,-0.00320,-0.2965,-0.3734,0.0405,-0.1825,0,1,0,1,62
1,1000246,1000246,NaN,NaN,-0.90395,NaN,1.10775,-0.26705,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,64
2,1000624,1000624,0.2874,-0.17145,0.42140,-0.08530,-0.25490,-0.74020,-2.1225,0.0602,...,-0.04580,-0.0013,-4.5999,0.2328,0.1268,0,1,0,0,61
3,1000950,1000950,NaN,NaN,1.25435,NaN,-0.08715,0.38445,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0,0,0,0,60
4,1002400,1002400,0.0484,0.10905,0.71500,0.21765,-0.09530,0.45640,0.5247,0.2293,...,0.33565,0.0697,-0.5116,-0.1965,-0.4389,1,0,0,1,61


In [64]:
olink9.shape

(28680, 2930)

In [65]:
olink9.to_csv('proteome_obesity_physact_md_age_sex_fin_march25.csv',sep='\t',index=False,na_rep='NaN')

In [66]:
!dx upload proteome_obesity_physact_md_age_sex_fin_march25.csv --path /zillur/bip_bmi_smd/pheno/

[===========================================================>] Uploaded 610,713,485 of 610,713,485 bytes (100%) proteome_obesity_physact_md_age_sex_fin_march25.csv
ID                                file-GzXZ1f8JpbBZ30kZBj62PpY4
Class                             file
Project                           project-GvFk9vjJpbBVBv78q23jKZ62
Folder                            /zillur/bip_bmi_smd/pheno
Name                              proteome_obesity_physact_md_age_sex_fin_march25.csv
State                             closing
Visibility                        visible
Types                             -
Properties                        -
Tags                              -
Outgoing links                    -
Created                           Thu Mar 27 12:13:13 2025
Created by                        zillur
 via the job                      job-GzXXQVjJpbByPk600kKyQPQ0
Last modified                     Thu Mar 27 12:13:15 2025
Media type                        
archivalState                     "